<h2 style="color: green;">데이터 인코딩</h2>

## 범주형 데이터

범주형 데이터는 연속적인 값을 갖지 않고 명목상으로 분류되는 값들을 의미합니다. 이러한 데이터는 숫자로 표현되지 않는다는 특징을 갖습니다.

범주형(categorical) 데이터는 연속형(numerical) 데이터와 다르게 숫자 값이 아니라 범주(category) 값으로 이루어 져 있습니다. 이러한 데이터는 대부분의 머신 러닝 알고리즘에 바로 적용할 수 없으며, 이를 처리하는 방법이 필 요합니다.

## 순서가 없는 범주형 데이터 처리

In [19]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

### *단일 클래스 특성에 대한 처리*

In [4]:
feature = np.array((['가나다라'],
                    ['가나다라'],
                    ['아바하자'],
                    ['카나다사']))
print(feature)

#원-핫 인코더 생성
one_hot = LabelBinarizer()  #객체 담기
one_hot.fit_transform(feature)  #특성을 원-핫 인코딩 변환
#특성 클래스 확인
print(one_hot.classes_)

[['가나다라']
 ['가나다라']
 ['아바하자']
 ['카나다사']]
['가나다라' '아바하자' '카나다사']


원-핫 인코딩은 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식입니다

### *다중 클래스 특성에 대한 처리*

In [5]:
multiclass_feature = [('거북이와 두루미', '가나다라마바'),
                     ('거북이와 토끼', '델몬트주스'),
                     ('거북이와 두루미', '아자바하나'),
                     ('토끼와 거북이', '카나다하바'),
                     ('콩쥐와 팥쥐', '아자바하나')
                     ]

one_hot_multi = MultiLabelBinarizer()
one_hot_multi.fit_transform(multiclass_feature)
print(one_hot_multi.classes_)

['가나다라마바' '거북이와 두루미' '거북이와 토끼' '델몬트주스' '아자바하나' '카나다하바' '콩쥐와 팥쥐' '토끼와 거북이']


### *문자열 타깃 데이터 원-핫 인코딩*

In [10]:
from sklearn.preprocessing import OneHotEncoder

str_feature = ([['안녕', 1],
                ['저녁', 2],
                ['안녕', 1],
                ['점심', 3],
               ])

one_hot_encoder = OneHotEncoder()

#One hot encoder -> 입력 특성 배열을 모두 범주형
one_hot_encoder.fit_transform(str_feature)
print(one_hot_encoder.categories_)

[array(['안녕', '저녁', '점심'], dtype=object), array([1, 2, 3], dtype=object)]


## 1-1 순서가 있는 범주형 특성 인코딩
+ 순서가 있는 클래스는 순서 개념을 가진 수치값으로 변환
+ 딕셔너리 사용해서 -> 특성

In [11]:
import pandas as pd

In [12]:
#특성 데이터 생성
dataframe = pd.DataFrame({
    'Score' : ['Low', 'Low', "Medium", "Medium", 'High']
})

print(dataframe)

    Score
0     Low
1     Low
2  Medium
3  Medium
4    High


In [13]:
#매핑 딕셔너리 생성
scale_mapper = {
    "Low" : 1,
    "Medium" : 2,
    "High" : 3
}

print(scale_mapper)

{'Low': 1, 'Medium': 2, 'High': 3}


In [14]:
data = dataframe['Score'].replace(scale_mapper)
print(data)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64


## 1-2 순서가 있는 범주형 특성 인코딩

In [15]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
feature_array = np.array((['Low', 10],
                        ['High', 40],
                        ['Medium', 30]
                        ))

ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit_transform(feature_array)
print(ordinal_encoder.categories_)

[array(['High', 'Low', 'Medium'], dtype='<U21'), array(['10', '30', '40'], dtype='<U21')]


## 1-3 순서가 있는 범주형 특성 인코딩
+ 특성 딕셔너리 인코딩

In [17]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
#딕셔너리 생성
data_dict = [{"Red" : 2, "Blue" : 4},
            {"Red" : 2, "Blue" : 3},
            {"Red" : 1, "Yellow" : 2},
            {"Red" : 1, "Yellow" : 2}]

dictVectorizer = DictVectorizer(sparse=False)
#sparse=False는 모든 데이터를 메모리에 올린다는 의미(밀집 행렬)

feature_dict = dictVectorizer.fit_transform(data_dict)
print(feature_dict)

feature_dict_name = dictVectorizer.get_feature_names_out()
print(feature_dict_name)

dict_data = pd.DataFrame(feature_dict, columns=feature_dict_name)
print(dict_data)

[[4. 2. 0.]
 [3. 2. 0.]
 [0. 1. 2.]
 [0. 1. 2.]]
['Blue' 'Red' 'Yellow']
   Blue  Red  Yellow
0   4.0  2.0     0.0
1   3.0  2.0     0.0
2   0.0  1.0     2.0
3   0.0  1.0     2.0


## 범주형 데이터 결측치 처리

### *누락된 클래스 값 대처하기 01*

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [28]:
x = np.array([[0, 2.10, 1.48],
             [1, 1.18, 1.33],
             [0, 1.22, 1.27],
             [1, -0.20, -1.15]])

x_with_nan = np.array([[np.nan, 0.87, 1.33], 
                       [np.nan, -0.67, -0.22]])
clf = KNeighborsClassifier(3, weights='distance')

print(x[:, 1:])  #feature
print(x[:,0])  #label

[[ 2.1   1.48]
 [ 1.18  1.33]
 [ 1.22  1.27]
 [-0.2  -1.15]]
[0. 1. 0. 1.]


In [27]:
train_model = clf.fit(x[:,1:], x[:,0])
imputer_values = train_model.predict(x_with_nan[:,1:])  #누락된 값의 클래스 예측

x_with_imputer = np.hstack((imputer_values.reshape(-1, 1), x_with_nan[:,1:]))
data = np.vstack((x_with_imputer, x))  #두 특성 행렬을 연결
print(data)

[[ 0.    0.87  1.33]
 [ 1.   -0.67 -0.22]
 [ 0.    2.1   1.48]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.2  -1.15]]


### *누락된 클래스 값 대처하기  02*
+ 누락된 값을 특성에서 가장 자주 등장하는 값으로 채우기

In [21]:
from sklearn.impute import SimpleImputer

In [22]:
x_complete = np.vstack((x_with_nan, x))

impute = SimpleImputer(strategy='most_frequent')
data_impute = impute.fit_transform(x_complete)
                                  
print(data_impute)

[[ 0.    0.87  1.33]
 [ 0.   -0.67 -0.22]
 [ 0.    2.1   1.48]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.2  -1.15]]
